# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [1]:
import json
from knowrob import *
    
InitKnowRob()

[14:34:18.959] [info] [KnowRob] static initialization done.


### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [2]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"},
            {"alias": "SOMA_DFL", "uri": "http://ease-crc.org/ont/SOMA_DFL.owl"}
		]
	},
	"data-sources": [
		{"path": "owl/USD.owl", "format": "rdf-xml"},
		{"path": "owl/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],  
    "reasoner": [
    {
      "type": "DummyReasoner",
      "name": "DummyReasoner",
      "module": "reasoner/dfl_reasoner.py",
      "data-backend": "redland"
    }
    ]
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

def runQuery(queryStr):
    phi = QueryParser.parse(queryStr)
    resultStream = kb.submitQuery(phi, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
    resultQueue = resultStream.createQueue()
    retq = []
    result = resultQueue.pop_front()
    while not result.indicatesEndOfEvaluation():
        if isinstance(result, AnswerYes):
            aux = {}
            for substitution in result.substitution():
                variable, term = substitution[1], substitution[2]
                aux[str(variable)] = str(term)
            retq.append(aux)
            result = resultQueue.pop_front()
    return retq


[14:34:30.199] [info] Using backend `redland` with type `Redland`.
[14:34:30.200] [info] [redland] using storage of type "memory" with name "knowrob" and options "contexts='yes'".
[14:34:30.200] [info] Using queryable backend with id 'redland'.
[14:34:30.200] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/rdf-schema.xml' with version "Mon Apr 22 16:37:36 2024" and origin "rdf-schema".
[14:34:30.201] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/owl.rdf' with version "Mon Apr 22 16:37:36 2024" and origin "owl".
[14:34:30.203] [info] Loading ontology at '/home/sascha/.knowrob/owl/USD.owl' with version "Wed Aug 14 13:42:10 2024" and origin "USD".
[14:34:30.208] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/external/DUL.owl' with version "02-09-2024" and origin "DUL".
[14:34:30.224] [info] Loading ontology at '/home/sascha/.knowrob/owl/house_4.owl' with version "Tue Aug 20 12:42:54 2024" and origin "house_4".
[14:34:30.224] [

### Submitting a Query

Now we want to run a query on the loaded semantic map:

In [4]:
# CQ1: which objects do I need for breakfast? -- reinterpreted as, what objects are subclasses of breakfast food, 
# and what can I use to serve those

# Are there any objects that are instances of breakfast food?
bdgs = runQuery("SOMA_DFL:isInstanceOf(?x, 'http://ease-crc.org/ont/SOMA_DFL.owl#breakfast_food.n.wn.food')")
breakfastFoodItems = [x["?x"] for x in bdgs]

# Are there any objects that contain something that is a breakfast food?

#     First, what kinds of breakfast food are there?
bdgs = runQuery("SOMA_DFL:isSubclassOf(?x, 'http://ease-crc.org/ont/SOMA_DFL.owl#breakfast_food.n.wn.food')")
breakfastFoodClasses = [x["?x"] for x in bdgs]

#     Second, what containers for these things are around?
containerItems = set()
breakfastFoodClassesPresent = set()
for food in breakfastFoodClasses:
    bdgs = runQuery("SOMA_DFL:hasPart(?x, '%s')" % food)
    containerItems = containerItems.union([x["?x"] for x in bdgs])
    if 0 < len(bdgs):
        breakfastFoodClassesPresent.add(food)

# What sort of tools are used to serve the breakfast food we found 
#     (either as stand-alone items, or contained somewhere)?
toolItems = set()
for food in set().union(breakfastFoodItems).union(breakfastFoodClassesPresent):
    bdgs = runQuery("SOMA_DFL:useMatch('http://ease-crc.org/ont/SOMA_DFL.owl#serve.v.wn.consumption..concrete', ?x, '%s')" % food)
    toolItems = toolItems.union([x["?x"] for x in bdgs])

# Report the results
edibles = containerItems.union(breakfastFoodItems)
print("These things look edible or contain something edible: %s" % str(sorted(edibles)))
print(" ... and might need these items to serve them: %s" % str(sorted(list(toolItems))))


# CQ2: Which objects contain something to drink?

# What kinds of beverages are there?
bdgs = runQuery("SOMA_DFL:isSubclassOf(?x, 'http://ease-crc.org/ont/SOMA_DFL.owl#beverage.n.wn.food')")
beverageClasses = [x["?x"] for x in bdgs]

# What containers for these things are around?
containerItems = set()
for drink in beverageClasses:
    bdgs = runQuery("SOMA_DFL:hasPart(?x, '%s')" % drink)
    containerItems = containerItems.union([x["?x"] for x in bdgs])

# Report results
print("These items should contain something to drink: %s" % str(sorted(list(containerItems))))

# CQ3: Where do we expect an item to be? -- note, there is no ranking of locations by likelihood,
# and there may be many plausible locations!

# For example, lets pick a tool from CQ1
item = list(toolItems)[0]

bdgs = runQuery("SOMA_DFL:useMatch('http://ease-crc.org/ont/SOMA_DFL.owl#store.v.wn.possession..place', ?x, '%s')" % item)
locations = [x["?x"] for x in bdgs]

# Report results
print("The %s might be in one of these places: %s" % (item, str(sorted(list(locations)))))

# CQ4: What can I grasp on an object to open it?

# CQ5: Where to place the utensils for breakfast?

bdgs = runQuery("SOMA_DFL:hasDisposition(?x, 'http://ease-crc.org/ont/SOMA_DFL.owl#serve.v.wn.consumption..concrete.Location')")
locations = [x["?x"] for x in bdgs]

# Report results
print("You could set up the meal at one of these locations: %s" % str(locations))


'''
phi1 = QueryParser.parse("SOMA_DFL:hasDisposition(?x, 'SOMA_DFL:cut.v.wn.contact.Instrument')")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))
'''

?y : USD:Bedroom_4


CQ1 Which objects do I need for breakfast/
CQ2 Which objects contain something to drink?
CQ3 Where do we expect an item to be?
CQ4 What can I grasp on an object to open it?
CQ5 Where should I put the utensils for breakfast?